# DATA CLEANING AND NORMALIZING NOTEBOOK

### **1. Import thư viện**

In [83]:
import os
import glob
import pandas as pd

### **2. Nối các file csv thu thập được thành một file duy nhất**

Thiết lập đường dẫn folder chứa các file csv input và tên file csv output sau khi nối

In [84]:
FOLDER = "data"  
OUTPUT_FILE = "football_players_full_dataset.csv"

In [85]:
def merge_csv(folder_path, output_path, drop_duplicate=True):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    if not all_files:
        print("Not found any CSV files in the folder.")
        return
    
    dfs = []

    for file in all_files:
        try:
            df = pd.read_csv(file)
            print(f"Loaded: {file} ({len(df)} rows)")
            dfs.append(df)
        except Exception as e:
            print(f"Read error for file {file}: {e}")

    merged_df = pd.concat(dfs, ignore_index=True)

    if drop_duplicate and "player_id" in merged_df.columns:
        merged_df.drop_duplicates(subset=["player_id"], inplace=True)

    merged_df.to_csv(output_path, index=False)
    print(f"\nComplete merging CSV files to {output_path}")
    print(f"Total players: {len(merged_df)}")

In [86]:
merge_csv(FOLDER, OUTPUT_FILE)

Loaded: data\A-League_Men.csv (378 rows)
Loaded: data\Allsvenskan.csv (523 rows)
Loaded: data\Argentine_Liga.csv (1274 rows)
Loaded: data\Austrian_Bundesliga.csv (390 rows)
Loaded: data\Belgian_Pro_League.csv (578 rows)
Loaded: data\Bolivian_Primera_División.csv (115 rows)
Loaded: data\Bundesliga.csv (578 rows)
Loaded: data\Canadian_Premier_League.csv (13 rows)
Loaded: data\Challenger_Pro_League.csv (515 rows)
Loaded: data\Chilean_Primera_División.csv (198 rows)
Loaded: data\Chinese_Super_League.csv (525 rows)
Loaded: data\Croatian_Football_League.csv (108 rows)
Loaded: data\Czech_First_League.csv (619 rows)
Loaded: data\Danish_Superliga.csv (159 rows)
Loaded: data\Eerste_Divisie.csv (25 rows)
Loaded: data\EFL_League_One.csv (270 rows)
Loaded: data\EFL_League_Two.csv (105 rows)
Loaded: data\Ekstraklasa.csv (2 rows)
Loaded: data\Eliteserien.csv (523 rows)
Loaded: data\Eredivisie.csv (638 rows)
Loaded: data\Hrvatska_NL.csv (415 rows)
Loaded: data\Indian_Super_League.csv (15 rows)
Loaded:

C:\Users\ACER\AppData\Local\Temp\ipykernel_25328\159325735.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(dfs, ignore_index=True)



Complete merging CSV files to football_players_full_dataset.csv
Total players: 20682


### **3. Lọc và lấy các dữ liệu liệu có giá trị**

Tên file kết quả

In [87]:
FINAL_OUTPUT = "football_players_dataset.csv"

Đọc dữ liệu từ file đã được nối

In [88]:
df = pd.read_csv(OUTPUT_FILE)
total_raw = len(df)

df["player_id"] = df["player_id"].astype(str)

Loại bỏ các cầu thủ thiếu giá trị chuyển nhượng

In [89]:
mv_col = "market_value"

df = df[~(
    df[mv_col].isna() |
    (df[mv_col].astype(str).str.strip() == "")
)]

print("- After removing missing market value:", len(df))

- After removing missing market value: 15567


Loại bỏ các cầu thủ bị trùng

In [90]:
before = len(df)

df = df.drop_duplicates(subset="player_id", keep="first")

after = len(df)
print("- Removed duplicate player_id:", before - after)
print("- After dedup:", after)

- Removed duplicate player_id: 0
- After dedup: 15567


Loại bỏ cầu thủ vi phạm logic

In [91]:
numeric_cols = [
    'age', 'height', 'goals', 'shots_per90',
    'minutes_played', 'shots_on_target_per90',
    'minutes_per_game', 'wins_per90', 'draws_per90', 'losses_per90'
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


invalid_mask = pd.Series(False, index=df.index, dtype=bool)

# 1. Tuổi (15–45)
if 'age' in df.columns:
    invalid_mask |= (df['age'] < 15) | (df['age'] > 45)

# 2. Chiều cao (150–220)
if 'height' in df.columns:
    invalid_mask |= (df['height'] < 150) | (df['height'] > 220)

# 3. goals ≤ total shots
if all(col in df.columns for col in ['goals', 'shots_per90', 'minutes_played']):
    total_shots = df['shots_per90'] * df['minutes_played'] / 90
    invalid_mask |= df['goals'] > total_shots

# 4. shots_on_target ≤ total shots
if all(col in df.columns for col in ['shots_on_target_per90', 'shots_per90']):
    invalid_mask |= df['shots_on_target_per90'] > df['shots_per90']

# 5. minutes per game ≤ 135
if 'minutes_per_game' in df.columns:
    invalid_mask |= df['minutes_per_game'] > 135

# 6. wins + draws + losses = 1
if all(col in df.columns for col in ['wins_per90', 'draws_per90', 'losses_per90']):
    total_results = df['wins_per90'] + df['draws_per90'] + df['losses_per90']
    invalid_mask |= (total_results - 1.0).abs() > 0.01


logic_bad_df = df.loc[invalid_mask].copy()
df = df.loc[~invalid_mask].copy().reset_index(drop=True)

print("- Removed logic-violated players:", len(logic_bad_df))
print("- After logic filter:", len(df))

- Removed logic-violated players: 429
- After logic filter: 15138


Loại bỏ các cầu thủ thiếu các trường dữ liệu quan trọng

In [92]:
GK_REQUIRED = [
    "goals_against_per90", "shots_on_target_against_per90", "saves_per90",
    "save_percentage", "clean_sheet_pct", "psxg_per_shot", "psxg_ga_per90",
    "penalty_save_pct", "passes_attempted_per90", "launch_pct",
    "avg_pass_length", "def_actions_outside_pen_per90",
    "avg_distance_def_actions", "crosses_stopped_pct"
]

DF_REQUIRED = [
    "tackles_per90", "interceptions_per90", "blocks_per90",
    "ball_recoveries_per90", "progressive_passes_per90",
    "progressive_carries_per90", "aerials_won_per90", "aerial_win_pct"
]

MF_REQUIRED = [
    "key_passes_per90", "passes_completed_per90", "progressive_passes_per90",
    "progressive_carries_per90", "sca_per90", "gca_per90",
    "tackles_per90", "interceptions_per90"
]

FW_REQUIRED = [
    "goals_per_90", "assists_per_90", "shots_per90",
    "shots_on_target_per90", "npxg_per90", "xag_per90",
    "npxg_xag_per90", "sca_per90", "gca_per90"
]

In [93]:
def get_group(position):
    position = str(position).upper()
    if "GK" in position:
        return "GK"
    if any(p in position for p in ["DF", "LB", "RB", "CB", "WB"]):
        return "DF"
    if any(p in position for p in ["MF", "DM", "CM", "AM"]):
        return "MF"
    if any(p in position for p in ["FW", "ST", "RW", "LW"]):
        return "FW"
    return None

clean_idx = []
unclean_idx = []

for idx, row in df.iterrows():
    group = get_group(row.get("position", ""))

    if group == "GK":
        required = GK_REQUIRED
    elif group == "DF":
        required = DF_REQUIRED
    elif group == "MF":
        required = MF_REQUIRED
    elif group == "FW":
        required = FW_REQUIRED
    else:
        unclean_idx.append(idx)
        continue

    missing = 0
    for col in required:
        if col not in df.columns or pd.isna(row.get(col)):
            missing += 1

    if missing == 0:
        clean_idx.append(idx)
    else:
        unclean_idx.append(idx)

clean_df = df.loc[clean_idx].reset_index(drop=True)
unclean_df = df.loc[unclean_idx].reset_index(drop=True)

Lưu kết quả cuối cùng

In [94]:
clean_df.to_csv(FINAL_OUTPUT, index=False)

print("\n  FINAL RESULT")
print("- Raw players:     ", total_raw)
print("- Total remaining: ", len(clean_df))
print("- Total removed:   ", total_raw - len(clean_df))


  FINAL RESULT
- Raw players:      20682
- Total remaining:  9680
- Total removed:    11002
